<a href="https://colab.research.google.com/github/hubertwel/paragraph-similarity/blob/main/paragraph-similarity/paragraph_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

seconds_in_a_day = 24 * 60 * 60
seconds_in_a_day

print('seconds_in_a_day: ', seconds_in_a_day)

seconds_in_a_day:  86400
